In [29]:
## WIP
## Get NN to work


"""Alak!  Rated E for Everyone.  It's a simple and amusing game.  Can be played without any players, with one player as well as two.

How to run the game:
1) Instantiate the class
2) Insert number of credits for number of players (0, 1, 2)
3) Choose your side (x, o)
4) Make your moves!

To run the testing method:
1) Alak.testing()

"""

from __future__ import print_function, division
from random import choice
import numpy as np
from IPython import display
import pickle
from colorama import Fore, Back, Style
from pdb import set_trace
import re

import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

np.set_printoptions(formatter={'float': '{:.5f}'.format})


class Alak:
    def __init__(self, pr=True, training=False, num_players=None):
        self.training = training
        self.num_players = num_players
        
        self.pr = pr
        
        self.step = []
        self.x_step = []     #Used to store moves that happen throughout the game.
        self.o_step = []
        
        self.board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])     #Game board.
        self.training_board = np.array([1, 1, 1, 1, 0, 0, -1, -1, -1, -1])     #Training game board
        
        self.test = ''     #Arbitrary initialization for the testing method.
        
        self.p1 = ''     #Arbitrary initialization to avoid future error if following if statement is not executed.
        self.p2 = ''     #Arbitrary initialization to avoid future error if following if statement is not executed.
        self.create_players()     #Will ask if you want 0, 1, or 2 players.
        
        self.turn = np.random.choice(['x', 'o'])     #Randomly chooses which side starts the game.
        if self.pr:
            print('{}-side starts the game!'.format(self.turn))
        
        #Self.defeat created for fun
        self.defeat = ['defeated', 'beaten to a pulp', 'crushed', 'overpowered', 'hulk-smashed', 'KO-ed', 'pummeled to pieces',\
                      'round-house-kicked in the face', 'chicken-dinnered by the winner', 'royal-flushed down the drain',\
                      "kamehameha'd", 'kill-Billed', 'blasted off at the speed of light', 'team-rocketed', 'shake-n-baked']
        
        self.read_x_pickle = []
        self.read_o_pickle = []
        
        self.multipickle_x = []
        self.multipickle_o = []
        
        self.winner = 1
        
        self.fight()     #Runs the game
        
    def pickle(self, read_x=None, read_o=None):
        if read_x != None:
            self.multipickle_x.extend(read_x)
            self.multipickle_o.extend(read_o)
            with open('Alak_pickle_x.p', 'w') as f:
                pickle.dump(self.multipickle_x, f)
#             self.read_x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))
            
            with open('Alak_pickle_o.p', 'w') as g:
                pickle.dump(self.multipickle_o, g)
#             self.read_o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
        else:
            with open('Alak_pickle_x.p', 'w') as f:
                pickle.dump(self.x_step, f)
            self.read_x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))

            with open('Alak_pickle_o.p', 'w') as g:
                pickle.dump(self.o_step, g)
            self.read_o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
        
         
    def create_players(self):     #Creates players with user interface.
        if self.training:
            num_players = self.num_players
        else:
            num_players = int(raw_input('Insert Credits:\n(0) Computer\n(1) Single Player\n(2) Two Players\n'))
        if num_players == 2:     #If 2, then the game initializes two players.
            self.p1 = raw_input('Please choose a side, x or o: ')
            if self.p1 == 'x':
                self.p2 = 'o'
            else:
                self.p2 = 'x'
            print('Player 1: {}-side!'.format(self.p1))
            print('Player 2: {}-side!'.format(self.p2))
        if num_players == 1:     #If 1, then creates only player 1.
            self.p1 = raw_input('Please choose a side, x or o: ')
            print('Player 1: {}-side!'.format(self.p1))
        
    def change_turn(self):     #Changes which side's turn it is.
        x, o = 'x', 'o'
        if self.turn == x:
            self.turn = o
        else:
            self.turn = x
    
    def convert_board(self):
        self.training_board = np.zeros(self.board.size)
        self.training_board[self.board=='x'] = 1
        self.training_board[self.board=='o'] = -1
        self.training_board[self.board=='_'] = 0
        
    def next_move(self):     #Determines the next move to make, either randomly for the computer or by choice of user.
        if self.turn != self.p1 and self.turn != self.p2:     #If not user, then computer moves self.turn side to random empty space.
            rand_empty = np.random.choice(np.where(self.board == '_')[0])     #Checks board for empty spaces, then chooses among all empty spaces.

            piece_pos = np.where(self.board == self.turn)[0]    #Indices of x/o pieces.
            rand_piece = np.random.choice(piece_pos)     #Among pieces, choose random piece to move to new, empty position, rand_empty.
            
            self.convert_board()
            before_board = self.training_board
            
            self.board[rand_empty], self.board[rand_piece] = self.board[rand_piece], self.board[rand_empty]     #Assign rand_piece to rand_empty.
            
            self.convert_board()
            after_board = self.training_board
            
            step = "{}: {} --> {}".format(self.turn, rand_piece, rand_empty)     #String that indicates which side's turn, moving rand_piece to rand_empty.
            self.step.append(step)
            if self.turn == 'x':
                self.x_step.append([np.append(before_board, after_board), 0])     #Appends step to self.x_step to store turn information.
#                 self.x_step.append([before_board, after_board, 0])
            else:
                self.o_step.append([before_board, after_board, 0])     #Either use a single array as X-input or use two arrays as X-input
#                 self.o_step.append([before_board, after_board, 0])

        else:     #User interface part.  Allows for user to choose particular piece and move to specified location.
            legal_spaces = np.where(self.board == '_')[0]     #Determines allowed spaces to move to.
            legal_pieces = np.where(self.board == self.turn)[0]     #Determines allow pieces to move.
            
            _break = True     #Initialized to break the following while loop if user inputs legal moves.
            while _break:
                pos_piece = int(raw_input("{}-side, select piece to move: ".format(self.turn)))     #User specifies piece to move.
                empty_space = int(raw_input("Move to: "))     #User specifies an empty space to move to.
                if pos_piece in legal_pieces and empty_space in legal_spaces:
                    self.convert_board()
                    before_board = self.training_board
                    
                    self.board[empty_space], self.board[pos_piece] = \
                        self.board[pos_piece], self.board[empty_space]     #Swaps empty space with piece chosen.
                        
                    self.convert_board()
                    after_board = self.training_board
                    
                    step = "{}: {} --> {}".format(self.turn, pos_piece, empty_space)
                    self.step.append(step)
                    if self.turn == 'x':
                        self.x_step.append([np.append(before_board, after_board), 0])     #Appends step to self.x_step to store turn information.
#                         self.x_step.append([before_board, after_board, 0])
                    else:
                        self.o_step.append([np.append(before_board, after_board), 0])
#                         self.o_step.append([before_board, after_board, 0])

                    _break = False
                else:     #Used to catch if user choices were illegal.
                    print("{} to {} is not a legal move, please choose among the legal moves."\
                                                                      .format(pos_piece, empty_space))
                    print("Legal Pieces: {}".format(legal_pieces))
                    print("Legal Spaces: {}".format(legal_spaces))
                    
    
    def check_dead(self, test=False, side='x'):     #Checks for kills made throughout the game.
        if test:     #Conditional for the testing method only.
            if side == 'x':
                dead_pat = re.search('xo+x', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('xo+x', ''.join(self.test))
                dead_pat = re.search('ox+o', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('ox+o', ''.join(self.test))
            else:
                dead_pat = re.search('ox+o', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('ox+o', ''.join(self.test))
                dead_pat = re.search('xo+x', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('xo+x', ''.join(self.test))
                
        if self.turn == 'x':     #Searches for kills during 'x' turn.
            dead_pat = re.search('xo+x', ''.join(self.board))     #Searches board for the regular expression pattern.
            while dead_pat != None:     #While loop to catch multiple matches.
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]     #Determines where the pattern starts and ends.
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')     #Replaces killed pieces on board.
                dead_pat = re.search('xo+x', ''.join(self.board))     #dead_pat reassigned until dead_pat returns None.
            dead_pat = re.search('ox+o', ''.join(self.board))     #Runs same code as above, but to check for suicide kills.
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('ox+o', ''.join(self.board))
        else:     #Searches for kills during 'o' turn.
            dead_pat = re.search('ox+o', ''.join(self.board))
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('ox+o', ''.join(self.board))
            dead_pat = re.search('xo+x', ''.join(self.board))
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('xo+x', ''.join(self.board))

    def fight(self):     #Actual start of game.
        if self.pr:
            print(*self.board)     #Print initial board
            print(*range(10))     #Print indices for easy location choices.
            print()
        Round = 1
        step_n = 0

        while len(np.unique(self.board)) > 2:     #Keeps game running until np.unique drops to 2 and below.
            if self.pr:
                print("Round: {}\n".format(Round))     #Prints current round.
            Round += 1
        ############################################
            for run in range(2):     #Used to run the following code twice in order to accommodate both sides.
                self.next_move()     #Determines which piece to move and moves it to an empty space
                
                if self.pr:
                    print(self.step[step_n])     #Prints move made.
                step_n += 1
                
                self.check_dead()     #Checks to see if a kill is made
                
                if self.pr:
                    print(*self.board)
                    print(*range(10))

                self.change_turn()     #Changes turn so opponent can make move.
                    
                if (self.board == 'x').sum() <= 1:     #Checks for game-over.
                    if self.pr:
                        print("Game over! x-side has {} piece(s) left! x-side has been {}!\n"\
                                .format((self.board == 'x').sum(), np.random.choice(self.defeat)))
                    self.o_step[-1][-1] = self.winner
                    self.pickle()
                    self.multipickle_x.extend(self.read_x_pickle)
                    self.multipickle_o.extend(self.read_o_pickle)
                    return
                if (self.board == 'o').sum() <= 1:     #Checks for game-over.
                    if self.pr:
                        print("Game over! o-side has {} piece(s) left! o-side has been {}!\n"\
                                  .format((self.board == 'o').sum(), np.random.choice(self.defeat)))
                    self.x_step[-1][-1] = self.winner
                    self.pickle()
                    self.multipickle_x.extend(self.read_x_pickle)
                    self.multipickle_o.extend(self.read_o_pickle)
                    return
            if self.pr:
                print()
    
    def testing_x(self):     #Tests "special" cases of Alak game (i.e. suicide, double-kill, etc.).
        passed = 0
        failed = 0
        special_cases = np.array([list('xoxoxx____'), list('xooxooxx__'), list('__xoo__oxo'), list('__xoo__xox')])
#         special_cases = np.array(list('oxoxoo____'), list('oxxoxxoo__'), list('_xoxx__ox_'), list('x__oxo_x__')])
        expected_results = np.array([list('x_x_xx____'), list('x__x__xx__'), list('__xoo__o_o'), list('__xoo__x_x')])
#         expected_results = np.array([list('o_o_oo____'), list('o__o__oo__'), list('_x_xx__ox_'), list('x__o_o_x__')])
        for i in range(len(special_cases)):
            self.test = np.array(special_cases[i])
            try:
                self.check_dead(test=True)
                assert np.array_equal(self.test, expected_results[i])
                print(self.test)
                print(expected_results[i])
            except:
                failed += 1
            else:
                passed += 1
        if failed != 0:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
            print(Fore.RED + "{} out of {} tests failed!".format(failed, len(special_cases)))
        else:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
        print(Style.RESET_ALL)
        
    def testing_o(self):     #Tests "special" cases of Alak game (i.e. suicide, double-kill, etc.).
        passed = 0
        failed = 0
#         special_cases = np.array([list('xoxoxx____'), list('xooxooxx__'), list('__xoo__oxo'), list('__xoo__xox')])
        special_cases = np.array([list('oxoxoo____'), list('oxxoxxoo__'), list('_xoxx__ox_'), list('x__oxo_x__')])
#         expected_results = np.array([list('x_x_xx____'), list('x__x__xx__'), list('__xoo__o_o'), list('__xoo__x_x')])
        expected_results = np.array([list('o_o_oo____'), list('o__o__oo__'), list('_x_xx__ox_'), list('x__o_o_x__')])
        for i in range(len(special_cases)):
            self.test = np.array(special_cases[i])
            try:
                self.check_dead(test=True, side='o')
                assert np.array_equal(self.test, expected_results[i])
                print(self.test)
                print(expected_results[i])
            except:
                failed += 1
            else:
                passed += 1
        if failed != 0:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
            print(Fore.RED + "{} out of {} tests failed!".format(failed, len(special_cases)))
        else:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
        print(Style.RESET_ALL)


# N = 50
Alak0 = Alak()
# for i in range(N-1):
#     Alak1 = Alak(pr=False, training=True, num_players=0)
#     Alak0.pickle(read_x = Alak1.read_x_pickle, read_o = Alak1.read_o_pickle)
    

# wins_x = 0
# wins_o = 0

# x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))
# o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
# print(pickle.load(open('Alak_pickle_x.p', 'r')))

# for elem in x_pickle:
#     if elem[1] == 1:
# #     if elem[2] == 1:
#         wins_x += 1
# for elem in o_pickle:
#     if elem[1] == 1:
# #     if elem[2] == 1:
#         wins_o += 1
# print(wins_x, wins_o)

# win_rate_x = wins_x/N
# win_rate_o = wins_o/N
# print(win_rate_x, win_rate_o)

# print(pickle.load(open('Alak_pickle_x.p', 'r')))
# print(pickle.load(open('Alak_pickle_o.p', 'r')))

# X = np.array([x_pickle[i][:1] for i, j in enumerate(x_pickle)])
# # print(X)
# y = np.array([x_pickle[i][1] for i, j in enumerate(x_pickle)])
# # print(y)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.25, random_state=0)
# print(X_train)
# print(y_train)
# print(np.atleast_2d(X_train).shape)
# # print(X_test)
# print(y_train.shape)
# # print(y_test)
# # print(len(X_train), len(X_test), len(y_train), len(y_test))

# clf = MLPClassifier(algorithm='l-bfgs', alpha=1e-10, hidden_layer_sizes=(10, 4), activation = 'tanh', \
#                     random_state=5, max_iter=1000000, learning_rate_init = 0.1)
# clf.fit(X_train, y_train)
# clf.decision_function(X_train)

Insert Credits:
(0) Computer
(1) Single Player
(2) Two Players
1
Please choose a side, x or o: o
Player 1: o-side!
x-side starts the game!
x x x x _ _ o o o o
0 1 2 3 4 5 6 7 8 9

Round: 1

x: 0 --> 5
_ x x x _ x o o o o
0 1 2 3 4 5 6 7 8 9
o-side, select piece to move: exit


ValueError: invalid literal for int() with base 10: 'exit'

In [26]:
Alak0.testing_o()
Alak0.testing_x()

['o' '_' 'o' '_' 'o' 'o' '_' '_' '_' '_']
['o' '_' 'o' '_' 'o' 'o' '_' '_' '_' '_']
['o' '_' '_' 'o' '_' '_' 'o' 'o' '_' '_']
['o' '_' '_' 'o' '_' '_' 'o' 'o' '_' '_']
['_' 'x' '_' 'x' 'x' '_' '_' 'o' 'x' '_']
['_' 'x' '_' 'x' 'x' '_' '_' 'o' 'x' '_']
['x' '_' '_' 'o' '_' 'o' '_' 'x' '_' '_']
['x' '_' '_' 'o' '_' 'o' '_' 'x' '_' '_']
4 out of 4 tests passed!

['x' '_' 'x' '_' 'x' 'x' '_' '_' '_' '_']
['x' '_' 'x' '_' 'x' 'x' '_' '_' '_' '_']
['x' '_' '_' 'x' '_' '_' 'x' 'x' '_' '_']
['x' '_' '_' 'x' '_' '_' 'x' 'x' '_' '_']
['_' '_' 'x' 'o' 'o' '_' '_' 'o' '_' 'o']
['_' '_' 'x' 'o' 'o' '_' '_' 'o' '_' 'o']
['_' '_' 'x' 'o' 'o' '_' '_' 'x' '_' 'x']
['_' '_' 'x' 'o' 'o' '_' '_' 'x' '_' 'x']
4 out of 4 tests passed!



In [85]:
## DON'T TOUCH
## NN works


"""Alak!  Rated E for Everyone.  It's a simple and amusing game.  Can be played without any players, with one player as well as two.

How to run the game:
1) Instantiate the class
2) Insert number of credits for number of players (0, 1, 2)
3) Choose your side (x, o)
4) Make your moves!

To run the testing method:
1) Alak.testing()

"""

from __future__ import print_function, division
from random import choice
import numpy as np
from IPython import display
import pickle
from colorama import Fore, Back, Style
from pdb import set_trace
import re

import sklearn
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.cross_validation import train_test_split

np.set_printoptions(formatter={'float': '{:.5f}'.format})


class Alak:
    def __init__(self, pr=True, training=False, num_players=None):
        self.training = training
        self.num_players = num_players
        
        self.pr = pr
        
        self.step = []
        self.x_step = []     #Used to store moves that happen throughout the game.
        self.o_step = []
        
        self.board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])     #Game board.
#         self.board = np.array(['x', '_', 'x', 'x', 'x', '_', 'o', 'o', 'o', 'o'])
        self.training_board = np.array([1, 1, 1, 1, 0, 0, -1, -1, -1, -1])     #Training game board
        
        self.x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))
        self.o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
        
        self.test = ''     #Arbitrary initialization for the testing method.
        
        self.p1 = ''     #Arbitrary initialization to avoid future error if following if statement is not executed.
        self.p2 = ''     #Arbitrary initialization to avoid future error if following if statement is not executed.
        self.create_players()     #Will ask if you want 0, 1, or 2 players.
        
#         self.turn = np.random.choice(['x', 'o'])     #Randomly chooses which side starts the game.
        self.turn = 'o'
        if self.pr:
            print('{}-side starts the game!'.format(self.turn))
        
        #Self.defeat created for fun
        self.defeat = ['defeated', 'beaten to a pulp', 'crushed', 'overpowered', 'hulk-smashed', 'KO-ed', 'pummeled to pieces',\
                      'round-house-kicked in the face', 'chicken-dinnered by the winner', 'royal-flushed down the drain',\
                      "kamehameha'd", 'kill-Billed', 'blasted off at the speed of light', 'team-rocketed', 'shake-n-baked']
        
        self.read_x_pickle = []
        self.read_o_pickle = []
        
        self.multipickle_x = []
        self.multipickle_o = []
        
        self.winner = 1
        
        self.fight()     #Runs the game
        
    def pickle(self, read_x=None, read_o=None):
        if read_x != None:
            self.multipickle_x.extend(read_x)
            self.multipickle_o.extend(read_o)
            with open('Alak_pickle_x.p', 'w') as f:
                pickle.dump(self.multipickle_x, f)
#             self.read_x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))
            
            with open('Alak_pickle_o.p', 'w') as g:
                pickle.dump(self.multipickle_o, g)
#             self.read_o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
        else:
            with open('Alak_pickle_x.p', 'w') as f:
                pickle.dump(self.x_step, f)
            self.read_x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))

            with open('Alak_pickle_o.p', 'w') as g:
                pickle.dump(self.o_step, g)
            self.read_o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
        
         
    def create_players(self):     #Creates players with user interface.
        if self.training:
            num_players = self.num_players
        else:
            num_players = int(raw_input('Insert Credits:\n(0) Computer\n(1) Single Player\n(2) Two Players\n'))
        if num_players == 2:     #If 2, then the game initializes two players.
            self.p1 = raw_input('Please choose a side, x or o: ')
            if self.p1 == 'x':
                self.p2 = 'o'
            else:
                self.p2 = 'x'
            print('Player 1: {}-side!'.format(self.p1))
            print('Player 2: {}-side!'.format(self.p2))
        if num_players == 1:     #If 1, then creates only player 1.
            self.p1 = raw_input('Please choose a side, x or o: ')
            print('Player 1: {}-side!'.format(self.p1))
        
    def change_turn(self):     #Changes which side's turn it is.
        x, o = 'x', 'o'
        if self.turn == x:
            self.turn = o
        else:
            self.turn = x
    
    def convert_board(self):
        self.training_board = np.zeros(self.board.size)
        self.training_board[self.board=='x'] = 1
        self.training_board[self.board=='o'] = -1
        self.training_board[self.board=='_'] = 0
        
    def next_move(self):     #Determines the next move to make, either randomly for the computer or by choice of user.
        if self.turn != self.p1 and self.turn != self.p2:     #If not user, then computer moves self.turn side to random empty space.
            self.convert_board()
            before_board = list(self.training_board)
            
            rand_empty = np.random.choice(np.where(self.board == '_')[0])     #Checks board for empty spaces, then chooses among all empty spaces.

            piece_pos = np.where(self.board == self.turn)[0]    #Indices of x/o pieces.
            rand_piece = np.random.choice(piece_pos)     #Among pieces, choose random piece to move to new, empty position, rand_empty.
            
            self.board[rand_empty], self.board[rand_piece] = self.board[rand_piece], self.board[rand_empty]     #Assign rand_piece to rand_empty.
            self.convert_board()
            after_board = list(self.training_board)
            
            step = "{}: {} --> {}".format(self.turn, rand_piece, rand_empty)     #String that indicates which side's turn, moving rand_piece to rand_empty.
            self.step.append(step)
            before_board.extend(after_board)
            if self.turn == 'x':
                self.x_step.append([before_board, 0])     #Appends step to self.x_step to store turn information.
#                 self.x_step.append([before_board, after_board, 0])
            else:
                self.o_step.append([before_board, 0])     #Either use a single array as X-input or use two arrays as X-input
#                 self.o_step.append([before_board, after_board, 0])

        else:     #User interface part.  Allows for user to choose particular piece and move to specified location.
            legal_spaces = np.where(self.board == '_')[0]     #Determines allowed spaces to move to.
            legal_pieces = np.where(self.board == self.turn)[0]     #Determines allow pieces to move.
            
            _break = True     #Initialized to break the following while loop if user inputs legal moves.
            while _break:
                pos_piece = int(raw_input("{}-side, select piece to move: ".format(self.turn)))     #User specifies piece to move.
                empty_space = int(raw_input("Move to: "))     #User specifies an empty space to move to.
                if pos_piece in legal_pieces and empty_space in legal_spaces:
                    self.convert_board()
                    before_board = list(self.training_board)
                    
                    self.board[empty_space], self.board[pos_piece] = \
                        self.board[pos_piece], self.board[empty_space]     #Swaps empty space with piece chosen.
                        
                    self.convert_board()
                    after_board = list(self.training_board)
                    
                    step = "{}: {} --> {}".format(self.turn, pos_piece, empty_space)
                    self.step.append(step)
                    before_board.extend(after_board)
                    if self.turn == 'x':
                        self.x_step.append([before_board, 0])     #Appends step to self.x_step to store turn information.
#                         self.x_step.append([before_board, after_board, 0])
                    else:
                        self.o_step.append([before_board, 0])
#                         self.o_step.append([before_board, after_board, 0])

                    _break = False
                else:     #Used to catch if user choices were illegal.
                    print("{} to {} is not a legal move, please choose among the legal moves."\
                                                                      .format(pos_piece, empty_space))
                    print("Legal Pieces: {}".format(legal_pieces))
                    print("Legal Spaces: {}".format(legal_spaces))
                    
    
    def check_dead(self, test=False, side='x'):     #Checks for kills made throughout the game.
        if test:     #Conditional for the testing method only.
            if side == 'x':
                dead_pat = re.search('xo+x', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('xo+x', ''.join(self.test))
                dead_pat = re.search('ox+o', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('ox+o', ''.join(self.test))
            else:
                dead_pat = re.search('ox+o', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('ox+o', ''.join(self.test))
                dead_pat = re.search('xo+x', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('xo+x', ''.join(self.test))
                
        if self.turn == 'x':     #Searches for kills during 'x' turn.
            dead_pat = re.search('xo+x', ''.join(self.board))     #Searches board for the regular expression pattern.
            while dead_pat != None:     #While loop to catch multiple matches.
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]     #Determines where the pattern starts and ends.
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')     #Replaces killed pieces on board.
                dead_pat = re.search('xo+x', ''.join(self.board))     #dead_pat reassigned until dead_pat returns None.
            dead_pat = re.search('ox+o', ''.join(self.board))     #Runs same code as above, but to check for suicide kills.
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('ox+o', ''.join(self.board))
        else:     #Searches for kills during 'o' turn.
            dead_pat = re.search('ox+o', ''.join(self.board))
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('ox+o', ''.join(self.board))
            dead_pat = re.search('xo+x', ''.join(self.board))
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('xo+x', ''.join(self.board))

    def fight(self):     #Actual start of game.
        if self.pr:
            print(*self.board)     #Print initial board
            print(*range(10))     #Print indices for easy location choices.
            print()
        Round = 1
        step_n = 0

        while len(np.unique(self.board)) > 2:     #Keeps game running until np.unique drops to 2 and below.
            if self.pr:
                print("Round: {}\n".format(Round))     #Prints current round.
            Round += 1
        ############################################
            for run in range(2):     #Used to run the following code twice in order to accommodate both sides.
                self.next_move()     #Determines which piece to move and moves it to an empty space
                
                if self.pr:
                    print(self.step[step_n])     #Prints move made.
                step_n += 1
                
                self.check_dead()     #Checks to see if a kill is made
                
                if self.pr:
                    print(*self.board)
                    print(*range(10))

                self.change_turn()     #Changes turn so opponent can make move.
                    
                if (self.board == 'x').sum() <= 1:     #Checks for game-over.
                    if self.pr:
                        print("Game over! x-side has {} piece(s) left! x-side has been {}!\n"\
                                .format((self.board == 'x').sum(), np.random.choice(self.defeat)))
                    self.o_step[-1][-1] = self.winner
                    self.pickle()
                    self.multipickle_x.extend(self.read_x_pickle)
                    self.multipickle_o.extend(self.read_o_pickle)
                    return
                if (self.board == 'o').sum() <= 1:     #Checks for game-over.
                    if self.pr:
                        print("Game over! o-side has {} piece(s) left! o-side has been {}!\n"\
                                  .format((self.board == 'o').sum(), np.random.choice(self.defeat)))
                    self.x_step[-1][-1] = self.winner
                    self.pickle()
                    self.multipickle_x.extend(self.read_x_pickle)
                    self.multipickle_o.extend(self.read_o_pickle)
                    return
            if self.pr:
                print()
    
    def testing_x(self):     #Tests "special" cases of Alak game (i.e. suicide, double-kill, etc.).
        passed = 0
        failed = 0
        special_cases = np.array([list('xoxoxx____'), list('xooxooxx__'), list('__xoo__oxo'), list('__xoo__xox')])
#         special_cases = np.array(list('oxoxoo____'), list('oxxoxxoo__'), list('_xoxx__ox_'), list('x__oxo_x__')])
        expected_results = np.array([list('x_x_xx____'), list('x__x__xx__'), list('__xoo__o_o'), list('__xoo__x_x')])
#         expected_results = np.array([list('o_o_oo____'), list('o__o__oo__'), list('_x_xx__ox_'), list('x__o_o_x__')])
        for i in range(len(special_cases)):
            self.test = np.array(special_cases[i])
            try:
                self.check_dead(test=True)
                assert np.array_equal(self.test, expected_results[i])
                print(self.test)
                print(expected_results[i])
            except:
                failed += 1
            else:
                passed += 1
        if failed != 0:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
            print(Fore.RED + "{} out of {} tests failed!".format(failed, len(special_cases)))
        else:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
        print(Style.RESET_ALL)
        
    def testing_o(self):     #Tests "special" cases of Alak game (i.e. suicide, double-kill, etc.).
        passed = 0
        failed = 0
#         special_cases = np.array([list('xoxoxx____'), list('xooxooxx__'), list('__xoo__oxo'), list('__xoo__xox')])
        special_cases = np.array([list('oxoxoo____'), list('oxxoxxoo__'), list('_xoxx__ox_'), list('x__oxo_x__')])
#         expected_results = np.array([list('x_x_xx____'), list('x__x__xx__'), list('__xoo__o_o'), list('__xoo__x_x')])
        expected_results = np.array([list('o_o_oo____'), list('o__o__oo__'), list('_x_xx__ox_'), list('x__o_o_x__')])
        for i in range(len(special_cases)):
            self.test = np.array(special_cases[i])
            try:
                self.check_dead(test=True, side='o')
                assert np.array_equal(self.test, expected_results[i])
                print(self.test)
                print(expected_results[i])
            except:
                failed += 1
            else:
                passed += 1
        if failed != 0:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
            print(Fore.RED + "{} out of {} tests failed!".format(failed, len(special_cases)))
        else:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
        print(Style.RESET_ALL)
        
    def NN_training(self, x_boards, o_boards, test_size=0.25):
        X_x = np.array([self.x_pickle[i][0] for i, j in enumerate(self.x_pickle)])
        y_x = np.array([self.x_pickle[i][1] for i, j in enumerate(self.x_pickle)])
        Xx_train, Xx_test, yx_train, yx_test = train_test_split(X_x, y_x, test_size=test_size, random_state=0)
        
        X_o = np.array([self.x_pickle[i][0] for i, j in enumerate(self.x_pickle)])
        y_o = np.array([self.x_pickle[i][1] for i, j in enumerate(self.x_pickle)])
        Xo_train, Xo_test, yo_train, yo_test = train_test_split(X_o, y_o, test_size=test_size, random_state=0)
        

        
        clf_x = MLPClassifier(algorithm='l-bfgs', alpha=1e-10, hidden_layer_sizes=(10, 4), activation = 'tanh',\
                    random_state=5, max_iter=1000000, learning_rate_init = 0.1)
        clf_x.fit(Xx_train, yx_train)
        clf_o = MLPClassifier(algorithm='l-bfgs', alpha=1e-10, hidden_layer_sizes=(10, 4), activation = 'tanh',\
                    random_state=5, max_iter=1000000, learning_rate_init = 0.1)
        clf_o.fit(Xo_train, yo_train)
        
#         x_decision = max(clf_x.decision_function())
#         o_decision = max(clf_o.decision_function())
#         return x_decision, o_decision
        


N = 2
Alak0 = Alak(pr=False, training=True, num_players=0)
for i in range(N-1):
    Alak1 = Alak(pr=False, training=True, num_players=0)
    Alak0.pickle(read_x = Alak1.read_x_pickle, read_o = Alak1.read_o_pickle)
    

wins_x = 0
wins_o = 0

x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))
o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
# print(pickle.load(open('Alak_pickle_x.p', 'r')))

for elem in x_pickle:
    if elem[1] == 1:
#     if elem[2] == 1:
        wins_x += 1
for elem in o_pickle:
    if elem[1] == 1:
#     if elem[2] == 1:
        wins_o += 1
print(wins_x, wins_o)

win_rate_x = wins_x/N
win_rate_o = wins_o/N
print(win_rate_x, win_rate_o)

# print(pickle.load(open('Alak_pickle_x.p', 'r')))
# print(pickle.load(open('Alak_pickle_o.p', 'r')))



0 2
0.0 1.0


In [3]:
## WIP
## Implement NN into code


"""Alak!  Rated E for Everyone.  It's a simple and amusing game.  Can be played without any players, with one player as well as two.

How to run the game:
1) Instantiate the class
2) Insert number of credits for number of players (0, 1, 2)
3) Choose your side (x, o)
4) Make your moves!

To run the testing method:
1) Alak.testing()

"""

from __future__ import print_function, division
from random import choice
import numpy as np
from IPython import display
import pickle
from colorama import Fore, Back, Style
from pdb import set_trace
import re

from sknn.mlp import Classifier, Layer

import sklearn
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.cross_validation import train_test_split

np.set_printoptions(formatter={'float': '{:.5f}'.format})


class Alak:
    def __init__(self, pr=True, training=False, num_players=None):
        self.training = training
        self.num_players = num_players
        
        self.pr = pr
        
        self.step = []
        self.x_step = []     #Used to store moves that happen throughout the game.
        self.o_step = []
        
        self.board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])     #Game board.
#         self.board = np.array(['x', '_', 'x', 'x', 'x', '_', 'o', 'o', 'o', 'o'])
        self.training_board = np.array([1, 1, 1, 1, 0, 0, -1, -1, -1, -1])     #Training game board
        
        self.x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))
        self.o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
        
        self.test = ''     #Arbitrary initialization for the testing method.
        
        self.p1 = ''     #Arbitrary initialization to avoid future error if following if statement is not executed.
        self.p2 = ''     #Arbitrary initialization to avoid future error if following if statement is not executed.
        self.create_players()     #Will ask if you want 0, 1, or 2 players.
        
#         self.turn = np.random.choice(['x', 'o'])     #Randomly chooses which side starts the game.
        self.turn = 'o'
        if self.pr:
            print('{}-side starts the game!'.format(self.turn))
        
        #Self.defeat created for fun
        self.defeat = ['defeated', 'beaten to a pulp', 'crushed', 'overpowered', 'hulk-smashed', 'KO-ed', 'pummeled to pieces',\
                      'round-house-kicked in the face', 'chicken-dinnered by the winner', 'royal-flushed down the drain',\
                      "kamehameha'd", 'kill-Billed', 'blasted off at the speed of light', 'team-rocketed', 'shake-n-baked']
        
        self.read_x_pickle = []
        self.read_o_pickle = []
        
        self.multipickle_x = []
        self.multipickle_o = []
        
        self.winner = 1
        
        self.fight()     #Runs the game
        
    def pickle(self, read_x=None, read_o=None):
        if read_x != None:
            self.multipickle_x.extend(read_x)
            self.multipickle_o.extend(read_o)
            with open('Alak_pickle_x.p', 'w') as f:
                pickle.dump(self.multipickle_x, f)
#             self.read_x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))
            
            with open('Alak_pickle_o.p', 'w') as g:
                pickle.dump(self.multipickle_o, g)
#             self.read_o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
        else:
            with open('Alak_pickle_x.p', 'w') as f:
                pickle.dump(self.x_step, f)
            self.read_x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))

            with open('Alak_pickle_o.p', 'w') as g:
                pickle.dump(self.o_step, g)
            self.read_o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
        
         
    def create_players(self):     #Creates players with user interface.
        if self.training:
            num_players = self.num_players
        else:
            num_players = int(raw_input('Insert Credits:\n(0) Computer\n(1) Single Player\n(2) Two Players\n'))
        if num_players == 2:     #If 2, then the game initializes two players.
            self.p1 = raw_input('Please choose a side, x or o: ')
            if self.p1 == 'x':
                self.p2 = 'o'
            else:
                self.p2 = 'x'
            print('Player 1: {}-side!'.format(self.p1))
            print('Player 2: {}-side!'.format(self.p2))
        if num_players == 1:     #If 1, then creates only player 1.
            self.p1 = raw_input('Please choose a side, x or o: ')
            print('Player 1: {}-side!'.format(self.p1))
        
    def change_turn(self):     #Changes which side's turn it is.
        x, o = 'x', 'o'
        if self.turn == x:
            self.turn = o
        else:
            self.turn = x
    
    def convert_board(self):
        self.training_board = np.zeros(self.board.size)
        self.training_board[self.board=='x'] = 1
        self.training_board[self.board=='o'] = -1
        self.training_board[self.board=='_'] = 0
        
    def next_move(self):     #Determines the next move to make, either randomly for the computer or by choice of user.
        if self.turn != self.p1 and self.turn != self.p2:     #If not user, then computer moves self.turn side to random empty space.
            self.convert_board()
            before_board = list(self.training_board)
            
            rand_empty = np.random.choice(np.where(self.board == '_')[0])     #Checks board for empty spaces, then chooses among all empty spaces.

            piece_pos = np.where(self.board == self.turn)[0]    #Indices of x/o pieces.
            rand_piece = np.random.choice(piece_pos)     #Among pieces, choose random piece to move to new, empty position, rand_empty.
            
            self.board[rand_empty], self.board[rand_piece] = self.board[rand_piece], self.board[rand_empty]     #Assign rand_piece to rand_empty.
            self.convert_board()
            after_board = list(self.training_board)
            
            step = "{}: {} --> {}".format(self.turn, rand_piece, rand_empty)     #String that indicates which side's turn, moving rand_piece to rand_empty.
            self.step.append(step)
            before_board.extend(after_board)
            if self.turn == 'x':
                self.x_step.append([before_board, 0])     #Appends step to self.x_step to store turn information.
#                 self.x_step.append([before_board, after_board, 0])
            else:
                self.o_step.append([before_board, 0])     #Either use a single array as X-input or use two arrays as X-input
#                 self.o_step.append([before_board, after_board, 0])

        else:     #User interface part.  Allows for user to choose particular piece and move to specified location.
            legal_spaces = np.where(self.board == '_')[0]     #Determines allowed spaces to move to.
            legal_pieces = np.where(self.board == self.turn)[0]     #Determines allow pieces to move.
            
            _break = True     #Initialized to break the following while loop if user inputs legal moves.
            while _break:
                pos_piece = int(raw_input("{}-side, select piece to move: ".format(self.turn)))     #User specifies piece to move.
                empty_space = int(raw_input("Move to: "))     #User specifies an empty space to move to.
                if pos_piece in legal_pieces and empty_space in legal_spaces:
                    self.convert_board()
                    before_board = list(self.training_board)
                    
                    self.board[empty_space], self.board[pos_piece] = \
                        self.board[pos_piece], self.board[empty_space]     #Swaps empty space with piece chosen.
                        
                    self.convert_board()
                    after_board = list(self.training_board)
                    
                    step = "{}: {} --> {}".format(self.turn, pos_piece, empty_space)
                    self.step.append(step)
                    before_board.extend(after_board)
                    if self.turn == 'x':
                        self.x_step.append([before_board, 0])     #Appends step to self.x_step to store turn information.
#                         self.x_step.append([before_board, after_board, 0])
                    else:
                        self.o_step.append([before_board, 0])
#                         self.o_step.append([before_board, after_board, 0])

                    _break = False
                else:     #Used to catch if user choices were illegal.
                    print("{} to {} is not a legal move, please choose among the legal moves."\
                                                                      .format(pos_piece, empty_space))
                    print("Legal Pieces: {}".format(legal_pieces))
                    print("Legal Spaces: {}".format(legal_spaces))
                    
    
    def check_dead(self, test=False, side='x'):     #Checks for kills made throughout the game.
        if test:     #Conditional for the testing method only.
            if side == 'x':
                dead_pat = re.search('xo+x', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('xo+x', ''.join(self.test))
                dead_pat = re.search('ox+o', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('ox+o', ''.join(self.test))
            else:
                dead_pat = re.search('ox+o', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('ox+o', ''.join(self.test))
                dead_pat = re.search('xo+x', ''.join(self.test))
                while dead_pat != None:
                    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.test))]
                    for elem in dead_se:
                        np.put(self.test, range(elem[0]+1, elem[1]-1), '_')
                    dead_pat = re.search('xo+x', ''.join(self.test))
                
        if self.turn == 'x':     #Searches for kills during 'x' turn.
            dead_pat = re.search('xo+x', ''.join(self.board))     #Searches board for the regular expression pattern.
            while dead_pat != None:     #While loop to catch multiple matches.
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]     #Determines where the pattern starts and ends.
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')     #Replaces killed pieces on board.
                dead_pat = re.search('xo+x', ''.join(self.board))     #dead_pat reassigned until dead_pat returns None.
            dead_pat = re.search('ox+o', ''.join(self.board))     #Runs same code as above, but to check for suicide kills.
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('ox+o', ''.join(self.board))
        else:     #Searches for kills during 'o' turn.
            dead_pat = re.search('ox+o', ''.join(self.board))
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('ox+o', ''.join(self.board))
            dead_pat = re.search('xo+x', ''.join(self.board))
            while dead_pat != None:
                dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(self.board))]
                for elem in dead_se:
                    np.put(self.board, range(elem[0]+1, elem[1]-1), '_')
                dead_pat = re.search('xo+x', ''.join(self.board))

    def fight(self):     #Actual start of game.
        if self.pr:
            print(*self.board)     #Print initial board
            print(*range(10))     #Print indices for easy location choices.
            print()
        Round = 1
        step_n = 0

        while len(np.unique(self.board)) > 2:     #Keeps game running until np.unique drops to 2 and below.
            if self.pr:
                print("Round: {}\n".format(Round))     #Prints current round.
            Round += 1
        ############################################
            for run in range(2):     #Used to run the following code twice in order to accommodate both sides.
                self.next_move()     #Determines which piece to move and moves it to an empty space
                
                if self.pr:
                    print(self.step[step_n])     #Prints move made.
                step_n += 1
                
                self.check_dead()     #Checks to see if a kill is made
                
                if self.pr:
                    print(*self.board)
                    print(*range(10))

                self.change_turn()     #Changes turn so opponent can make move.
                    
                if (self.board == 'x').sum() <= 1:     #Checks for game-over.
                    if self.pr:
                        print("Game over! x-side has {} piece(s) left! x-side has been {}!\n"\
                                .format((self.board == 'x').sum(), np.random.choice(self.defeat)))
                    self.o_step[-1][-1] = self.winner
                    self.pickle()
                    self.multipickle_x.extend(self.read_x_pickle)
                    self.multipickle_o.extend(self.read_o_pickle)
                    return
                if (self.board == 'o').sum() <= 1:     #Checks for game-over.
                    if self.pr:
                        print("Game over! o-side has {} piece(s) left! o-side has been {}!\n"\
                                  .format((self.board == 'o').sum(), np.random.choice(self.defeat)))
                    self.x_step[-1][-1] = self.winner
                    self.pickle()
                    self.multipickle_x.extend(self.read_x_pickle)
                    self.multipickle_o.extend(self.read_o_pickle)
                    return
            if self.pr:
                print()
    
    def testing_x(self):     #Tests "special" cases of Alak game (i.e. suicide, double-kill, etc.).
        passed = 0
        failed = 0
        special_cases = np.array([list('xoxoxx____'), list('xooxooxx__'), list('__xoo__oxo'), list('__xoo__xox')])
#         special_cases = np.array(list('oxoxoo____'), list('oxxoxxoo__'), list('_xoxx__ox_'), list('x__oxo_x__')])
        expected_results = np.array([list('x_x_xx____'), list('x__x__xx__'), list('__xoo__o_o'), list('__xoo__x_x')])
#         expected_results = np.array([list('o_o_oo____'), list('o__o__oo__'), list('_x_xx__ox_'), list('x__o_o_x__')])
        for i in range(len(special_cases)):
            self.test = np.array(special_cases[i])
            try:
                self.check_dead(test=True)
                assert np.array_equal(self.test, expected_results[i])
                print(self.test)
                print(expected_results[i])
            except:
                failed += 1
            else:
                passed += 1
        if failed != 0:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
            print(Fore.RED + "{} out of {} tests failed!".format(failed, len(special_cases)))
        else:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
        print(Style.RESET_ALL)
        
    def testing_o(self):     #Tests "special" cases of Alak game (i.e. suicide, double-kill, etc.).
        passed = 0
        failed = 0
#         special_cases = np.array([list('xoxoxx____'), list('xooxooxx__'), list('__xoo__oxo'), list('__xoo__xox')])
        special_cases = np.array([list('oxoxoo____'), list('oxxoxxoo__'), list('_xoxx__ox_'), list('x__oxo_x__')])
#         expected_results = np.array([list('x_x_xx____'), list('x__x__xx__'), list('__xoo__o_o'), list('__xoo__x_x')])
        expected_results = np.array([list('o_o_oo____'), list('o__o__oo__'), list('_x_xx__ox_'), list('x__o_o_x__')])
        for i in range(len(special_cases)):
            self.test = np.array(special_cases[i])
            try:
                self.check_dead(test=True, side='o')
                assert np.array_equal(self.test, expected_results[i])
                print(self.test)
                print(expected_results[i])
            except:
                failed += 1
            else:
                passed += 1
        if failed != 0:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
            print(Fore.RED + "{} out of {} tests failed!".format(failed, len(special_cases)))
        else:
            print("{} out of {} tests passed!".format(passed, len(special_cases)))
        print(Style.RESET_ALL)
        
    def NN_training(self, x_boards, o_boards, test_size=0.25):
        X_x = np.array([self.x_pickle[i][0] for i, j in enumerate(self.x_pickle)])
        y_x = np.array([self.x_pickle[i][1] for i, j in enumerate(self.x_pickle)])
        Xx_train, Xx_test, yx_train, yx_test = train_test_split(X_x, y_x, test_size=test_size, random_state=0)
        
        X_o = np.array([self.x_pickle[i][0] for i, j in enumerate(self.x_pickle)])
        y_o = np.array([self.x_pickle[i][1] for i, j in enumerate(self.x_pickle)])
        Xo_train, Xo_test, yo_train, yo_test = train_test_split(X_o, y_o, test_size=test_size, random_state=0)
        

        
        clf_x = MLPClassifier(algorithm='l-bfgs', alpha=1e-10, hidden_layer_sizes=(10, 4), activation = 'tanh',\
                    random_state=5, max_iter=1000000, learning_rate_init = 0.1)
        clf_x.fit(Xx_train, yx_train)
        clf_o = MLPClassifier(algorithm='l-bfgs', alpha=1e-10, hidden_layer_sizes=(10, 4), activation = 'tanh',\
                    random_state=5, max_iter=1000000, learning_rate_init = 0.1)
        clf_o.fit(Xo_train, yo_train)
        
#         x_decision = max(clf_x.decision_function())
#         o_decision = max(clf_o.decision_function())
#         return x_decision, o_decision
        


N = 2
Alak0 = Alak(pr=False, training=True, num_players=0)
for i in range(N-1):
    Alak1 = Alak(pr=False, training=True, num_players=0)
    Alak0.pickle(read_x = Alak1.read_x_pickle, read_o = Alak1.read_o_pickle)
    

wins_x = 0
wins_o = 0

x_pickle = pickle.load(open('Alak_pickle_x.p', 'r'))
o_pickle = pickle.load(open('Alak_pickle_o.p', 'r'))
# print(pickle.load(open('Alak_pickle_x.p', 'r')))

for elem in x_pickle:
    if elem[1] == 1:
#     if elem[2] == 1:
        wins_x += 1
for elem in o_pickle:
    if elem[1] == 1:
#     if elem[2] == 1:
        wins_o += 1
print(wins_x, wins_o)

win_rate_x = wins_x/N
win_rate_o = wins_o/N
print(win_rate_x, win_rate_o)

# print(pickle.load(open('Alak_pickle_x.p', 'r')))
# print(pickle.load(open('Alak_pickle_o.p', 'r')))



ImportError: cannot import name MLPRegressor